# Data Preparation for ML Engine

In this notebook we prepare the bank marketing data for our model by splitting it into training, evaluation, and testing sets.

### Read the data from Cloud Storage

In [1]:
# Get the project id
import os
project_id = os.environ['VM_PROJECT']

In [2]:
# Read the data from Cloud Storage
%gcs read -o gs://$project_id-mlengine/data/bank-additional-full.csv -v data_file

In [3]:
# Import necessary libraries for handling the data
import pandas as pd
import numpy as np
from io import BytesIO

In [4]:
# Read the data into pandas DataFrame
data = pd.read_csv(BytesIO(data_file), sep=';')

### Inspect the data

In [5]:
# Inspect the first rows
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [6]:
# Inpsect the info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
age               41188 non-null int64
job               41188 non-null object
marital           41188 non-null object
education         41188 non-null object
default           41188 non-null object
housing           41188 non-null object
loan              41188 non-null object
contact           41188 non-null object
month             41188 non-null object
day_of_week       41188 non-null object
duration          41188 non-null int64
campaign          41188 non-null int64
pdays             41188 non-null int64
previous          41188 non-null int64
poutcome          41188 non-null object
emp.var.rate      41188 non-null float64
cons.price.idx    41188 non-null float64
cons.conf.idx     41188 non-null float64
euribor3m         41188 non-null float64
nr.employed       41188 non-null float64
y                 41188 non-null object
dtypes: float64(5), int64(5), object(11)
memory usa

In [7]:
# View the target distribution
data['y'].value_counts()

no     36548
yes     4640
Name: y, dtype: int64

In [8]:
nb_yes = data['y'].value_counts()['yes']
print('Percentage of positive answers in dataset: {} %'.format(np.round(100 * nb_yes / len(data),2)))

Percentage of positive answers in dataset: 11.27 %


### Balance data by undersampling

In [9]:
# Drop 70% of the calls with negative answers
data_no = data[data['y'] == 'no'].sample(frac=0.3)
data_yes = data[data['y'] == 'yes']
data = pd.concat([data_no, data_yes]).sample(frac=1.0)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15604 entries, 24075 to 8680
Data columns (total 21 columns):
age               15604 non-null int64
job               15604 non-null object
marital           15604 non-null object
education         15604 non-null object
default           15604 non-null object
housing           15604 non-null object
loan              15604 non-null object
contact           15604 non-null object
month             15604 non-null object
day_of_week       15604 non-null object
duration          15604 non-null int64
campaign          15604 non-null int64
pdays             15604 non-null int64
previous          15604 non-null int64
poutcome          15604 non-null object
emp.var.rate      15604 non-null float64
cons.price.idx    15604 non-null float64
cons.conf.idx     15604 non-null float64
euribor3m         15604 non-null float64
nr.employed       15604 non-null float64
y                 15604 non-null object
dtypes: float64(5), int64(5), object(11)
memory 

In [11]:
# View the target distribution
data['y'].value_counts()

no     10964
yes     4640
Name: y, dtype: int64

In [12]:
nb_yes = data['y'].value_counts()['yes']
print('Percentage of positive answers in dataset: {} %'.format(np.round(100 * nb_yes / len(data),2)))

Percentage of positive answers in dataset: 29.74 %


### Split the data into training and evaluation sets

In [13]:
# Drop unnecessary columns
data.drop(columns='duration', inplace=True)

In [14]:
# Import a splitting tool
from sklearn.model_selection import train_test_split

# Split the data into train, evaluation, and test sets in a stratified manner
data_train_eval, data_test = train_test_split(data, test_size=0.01, stratify=data['y'])
data_train, data_eval = train_test_split(data_train_eval, test_size=0.3, stratify=data_train_eval['y'])

In [15]:
nb_yes_train = data_train['y'].value_counts()['yes']
nb_yes_eval = data_eval['y'].value_counts()['yes']

In [16]:
print('Percentage of positive answers in training set: {} %'.format(np.round(100 * nb_yes_train / len(data_train),2)))
print('Percentage of positive answers in evaluation set: {} %'.format(np.round(100 * nb_yes_eval / len(data_eval),2)))

Percentage of positive answers in training set: 29.74 %
Percentage of positive answers in evaluation set: 29.73 %


### Write into CSV and copy to Cloud Storage

In [17]:
# Write to CSV without index or headers
data_train.to_csv('bank_data_train.csv', index=False, header=False)
data_eval.to_csv('bank_data_eval.csv', index=False, header=False)
data_test.to_csv('bank_data_test.csv', index=False, header=False)

In [18]:
# Move prepared data to storage bucket
!gsutil mv bank_data*.csv gs://$project_id-mlengine/data/



Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update

Copying file://bank_data_eval.csv [Content-Type=text/csv]...
Removing file://bank_data_eval.csv...                                           
Copying file://bank_data_test.csv [Content-Type=text/csv]...
Removing file://bank_data_test.csv...                                           
Copying file://bank_data_train.csv [Content-Type=text/csv]...
Removing file://bank_data_train.csv...                                          

Operation completed over 3 objects/1.8 MiB.                                      
